In [1]:
from manim import *
import jupyter_capture_output

video_scene = " -v WARNING  --disable_caching phase_group_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s phase_group_Scene"

Jupyter Capture Output v0.0.11


In [116]:
# phase velocity
v_ph = 10

# amplitudes
A = 4


# dispersion
def omega(k):
    return k**2 * v_ph


# wave vectors
k_1 = 1.0
k_2 = 1.2

# angular frequencies
omega_1 = omega(k_1)
omega_2 = omega(k_2)

# averages
avg_k = (k_1 + k_2) / 2
avg_omega = (omega_1 + omega_2) / 2

# differences
delta_k = (k_2 - k_1) / 2
delta_omega = (omega_2 - omega_1) / 2


# wavepackage
def get_wavepackage(t):
    def wavepackage(x):
        xi_1 = A * np.cos(k_1*x - omega_1*t)
        xi_2 = A * np.cos(k_2*x - omega_2*t)
        return xi_1 + xi_2
    return wavepackage


# evelope of the wavepackage
def get_envelope(t, sig = 1):
    def envelope(x):
        return sig * 2*A * np.cos(delta_k*x - delta_omega*t)
    return envelope


# positions of the v_ph and v_gr dots
def get_x_v_ph(t):
    return avg_omega*t / avg_k + np.pi/delta_k

def get_x_v_gr(t):
    return delta_omega*t / delta_k + np.pi/2/delta_k 


# velocities
v_ph = avg_omega / avg_k
v_gr = delta_omega / delta_k

print(f"phase velocity: {v_ph}")
print(f"group velocity: {v_gr}")

phase velocity: 11.09090909090909
group velocity: 21.999999999999996


In [129]:
%%manim -qm --fps 60 $video_scene

class phase_group_Scene(Scene):
	def construct(self):
		CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
		self.add(CVC)

		# headline
		healine_phase_group = Title(r"Phasen- und Gruppengeschwindigkeit", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT) 
		healine_phase_group[0][0:6].set_color(RED)
		healine_phase_group[0][10:17].set_color(BLUE)
		self.add(healine_phase_group)


		# create ax
		x_length = 10
		y_length = 4
		x_range = [0, 200, 2]
		y_range = [-10, 10, 2]

		ax = Axes(x_length = x_length, y_length = y_length, x_range = x_range, y_range = y_range, tips = True, axis_config = {"include_ticks": False}).move_to([0, -0.5, 0])
		ax_xlabel = ax.get_x_axis_label(Tex(r"$x$", font_size = 36, color = WHITE))
		ax_ylabel = ax.get_y_axis_label(Tex(r"$\xi(x, t)$", font_size = 36, color = WHITE))
		self.add(ax, ax_xlabel, ax_ylabel)


		# create wave and envelope
		wave = ax.plot(get_wavepackage(0), color = GRAY, x_range = [0, 190])
		envelope_top = ax.plot(get_envelope(0), color = GRAY, stroke_width = 1.5, x_range = [0, 190])
		envelope_down = ax.plot(get_envelope(0, -1), color = GRAY, stroke_width = 1.5, x_range = [0, 190])
		self.add(wave)


		# v_ph and v_gr dots
		v_ph_dot = Dot(point = ax.c2p(get_x_v_ph(0), get_wavepackage(0)(get_x_v_ph(0))), color = RED, radius = 0.08)
		v_gr_dot = Dot(point = ax.c2p(get_x_v_gr(0), get_envelope(0)(get_x_v_gr(0))), color = BLUE, radius = 0.08)
		self.add(v_ph_dot, v_gr_dot)


		# wave updater
		def wave_updater(wave):
			t = t_tracker.get_value()
			# envelope
			new_envelope_top = ax.plot(get_envelope(t), color = GRAY, stroke_width = 1.5, x_range = [0, 190])
			new_envelope_down = ax.plot(get_envelope(t, -1), color = GRAY, stroke_width = 1.5, x_range = [0, 190])
			envelope_top.become(new_envelope_top)
			envelope_down.become(new_envelope_down)
			# dots
			new_v_ph_dot = Dot(point = ax.c2p(get_x_v_ph(t), get_wavepackage(t)(get_x_v_ph(t))), color = RED, radius = 0.08)
			new_v_gr_dot = Dot(point = ax.c2p(get_x_v_gr(t), get_envelope(t)(get_x_v_gr(t))), color = BLUE, radius = 0.08)
			v_ph_dot.become(new_v_ph_dot)
			v_gr_dot.become(new_v_gr_dot)
			new_wave = ax.plot(get_wavepackage(t), color = GRAY, x_range = [0, 190],)
			wave.become(new_wave)


		t_tracker = ValueTracker(0)
		wave.add_updater(wave_updater)

		self.wait(1)
		self.play(t_tracker.animate.set_value(2.5), rate_func = linear, run_time = 3)
		self.play(t_tracker.animate.set_value(5), rate_func = linear, run_time = 3)
		self.play(t_tracker.animate.set_value(7.5), rate_func = linear, run_time = 3)
		self.wait(3)

Manim Community v0.18.1